# Lista 5
## Dominik Frankowski
Poniższy dokument jest podzielony na sekcje:
1. funkcje -  implementacje algorytmów opisanych w dalszej części
    - pomocnicze
    - eliminacja Gaussa
    - eliminacja Gaussa z częściowym wyborem
    - rozkład LU (metodą eliminacji Gaussa)
    - rozkład LU (metodą eliminacji Gaussa z częściowym wyborem)
2. sprawozdanie - opis problemu, analiza i rozwiązanie
    - zadanie
    - rozwiązanie: opis, analiza i implementacja
3. testy
    - test manualny (do własnej edycji)
    - test poprawności
    - testy wydajności
    - analiza i wnioski

# 1. Funkcje
(Nad każdą funkcją znajduje się jej opis)

##  Pomocnicze
(odczyt i zapis z/do pliku, generowanie macierzy i wektora)

In [1]:
using SparseArrays
using LinearAlgebra
include("matrixgen.jl")

"""
Funkcja wczytania macierzy z pliku
"""
function load_matrix_from_file(file_path::String)
    toRet = nothing
    open(file_path) do file
        firstLine = split(readline(file))
        n = parse(Int64, firstLine[1])
        l = parse(Int64, firstLine[2])
        A = spzeros(n, n) #Sparse matrix wypełniony zerami
        for line in eachline(file)
            data = split(line, " ")
            i = parse(Int64, data[1])
            j = parse(Int64, data[2])
            value = parse(Float64, data[3])
            A[j, i] = value
        end
        toRet = (A, n, l)
    end
    return toRet
end

"""
Funkcja wczytania wektora (prawych stron) b z pliku
"""
function load_b_from_file(file_path::String)
    toRet = nothing
    open(file_path, "r") do file
        n = parse(Int64, readline(file))
        # Wektor b
        b = Vector{Float64}(undef, n)
        i = 1
        for line in eachline(file)
            b[i] = parse(Float64, line)
            i += 1
        end
        toRet = (b, n)
    end
    return toRet
end

"""
Funkcja zapisania macierzy wynikowej (z możliwością zapisu błędu względnego) do pliku
"""
function write_results_to_file(file_path::String, x::Vector{Float64}, n::Int64, is_b_generated::Bool)
    open(file_path, "w") do file
        if is_b_generated 
            x_ones = ones(n)
            blad_wzgledny::Float64 = norm(x_ones - x) / norm(x)
            println(file, blad_wzgledny)
        end

        for i in 1:n
            println(file, x[i])
        end
    end
end

"""
Funkcja generowania wektora prawych stron, gdy x=(1,...,1)^T
"""
function generate_b_vector(A::SparseMatrixCSC{Float64, Int64}, n::Int64, l::Int64)
    b = zeros(Float64, n)
    start_column = 1
    end_column = l
    for i in 1:n
        # dodanie wartości z macierzy Ak i Bk
        for j in start_column:end_column
            b[i] += A[j, i]
        end
        # dodanie wartości z macierzy Ck
        if i + l <= n
            b[i] += A[i + l, i]
        end
        # przesunięcie iteratora na kolejny blok
        if i % l == 0
            start_column = convert(Int64, (l * (i / l)))
            end_column += l
        end
    end
    return b
end

generate_b_vector

## 1) Eliminacja Gaussa

In [2]:
"""
Funkcja przekształcenia macierzy A i wektora b metodą eliminacji Gaussa
"""
function gauss_elimination_on_sparse_matrix(A::SparseMatrixCSC{Float64, Int64}, b::Vector{Float64}, 
        n::Int64, l::Int64)
    # Iteracja po kolumnach
    for k in 1:n-1
        # Iteracja po (określonych) wierszach kolumny k 
        for i in k+1:k+(l-k%l)
            multiplier = A[k, i] / A[k, k]
            A[k, i] = Float64(0.0)
            # Iteracja po (określonych) kolumnach w aktualnie eliminowanym wierszu
            # czyli inaczej operacja odejmowania wiersza od wiersza
            for j in k+1:min(k+l, n)
                A[j, i] -= multiplier * A[j, k]
            end
            b[i] -= multiplier * b[k]
        end
    end
    return (A, b)
end

"""
Funkcja obliczenia wektora X z macierzy A i wektora b metodą eliminacji Gaussa
"""
function solve_sparse_matrix_with_gauss_elimination(A::SparseMatrixCSC{Float64, Int64}, b::Vector{Float64}, 
        n::Int64, l::Int64)
    A_res, b_res = gauss_elimination_on_sparse_matrix(A, b, n, l)

    # Wektor rozwiązań X
    x = Vector{Float64}(undef,n)
    
    ## Wyliczenie wektora X
    # Iteracja po wierszach (od dołu)
    for i in n:-1:1
        row_sum = Float64(0.0)

        # Iteracja po kolumnach
        for j in i+1:min(n, i + l)
            row_sum += A_res[j, i] * x[j]
        end
        x[i] = (b_res[i] - row_sum) / A_res[i, i]
    end

    return x
end

solve_sparse_matrix_with_gauss_elimination

## 2) Eliminacja Gaussa z częściowym wyborem

In [3]:
"""
Funkcja przekształcenia macierzy i wektora b metodą eliminacji Gaussa z częściowym wyborem
"""
function gauss_elimination_with_choose_on_sparse_matrix(A::SparseMatrixCSC{Float64, Int64}, b::Vector{Float64}, 
        n::Int64, l::Int64)
    # Wektor permutacji, czyli zamiana wierszy przy częsciowym wyborze, 
    # ale bez "fizycznej" zamiany wierszy w macierzy 
    permutation = Vector{Int64}(undef,n)
    # Podstawowa permutacja
    for i in 1:n
        permutation[i] = i
    end
    # Iteracja po kolumnach
    for k in 1:n-1
        # Maksymalna wartość w danym wierszu
        max_element = abs(A[k, k])
        # Numer wiersza z największą wartością
        max_element_row = k
        for i in k:k+(l-k%l)
            # Znalezienie największej wartości oraz jej numeru wiersza
            if abs(A[k, permutation[i]]) > max_element
                max_element = abs(A[k, permutation[i]])
                max_element_row = i
            end
        end
        # Swap wierszy
        permutation[k], permutation[max_element_row] = permutation[max_element_row], permutation[k]
        # Iteracja po wierszach w kolumnie (tych, które są niezerowe i należy je "usunąć")
        for i in k+1:k+(l-k%l)
            multiplier = A[k, permutation[i]] / A[k, permutation[k]]
            # Zerowanie elementu eliminowanego
            A[k, permutation[i]] = Float64(0.0)
            # Iteracja po kolumnach (czyli inaczej odejmowanie wiersza od wiersza)
            for j in k+1:min(k + 2*l, n)
                A[j, permutation[i]] -= multiplier * A[j, permutation[k]]
            end
            b[permutation[i]] -= multiplier * b[permutation[k]]
        end 
    end
    return (A, permutation, b)
end

"""
Funkcja obliczenia wektora X z macierzy A i wektora b metodą eliminacji Gaussa z częściowym wyborem
"""
function solve_sparse_matrix_with_gauss_elimination_with_choose(A::SparseMatrixCSC{Float64, Int64}, 
        b::Vector{Float64}, n::Int64, l::Int64)
    A_res, permutation_res, b_res = gauss_elimination_with_choose_on_sparse_matrix(A, b, n, l)

    # Wektor rozwiązań X
    x = Vector{Float64}(undef,n)

    ## Obliczenie wektora X  startując "od dołu"
    # Iteracja po wierszach
    for i in n:-1:1
        row_sum = Float64(0.0)
        # Iteracja po kolumnach
        for j in i+1:min(i + 2*l + 1, n)
            row_sum += A_res[j, permutation_res[i]] * x[j]
        end
        x[i] = (b_res[permutation_res[i]] - row_sum) / A_res[i, permutation_res[i]]
    end
    return x
end

solve_sparse_matrix_with_gauss_elimination_with_choose

## 3) Rozkład LU (metodą eliminacji Gaussa)

In [4]:
"""
Funkcja przekształcenia macierzy A metodą rozkładu LU (przy użyciu eliminacji Gaussa)
"""
function lu_on_sparse_matrix(A::SparseMatrixCSC{Float64, Int64}, n::Int64, l::Int64)
    # Iteracja po kolumnach
    for k in 1:n-1
        # Iteracja po (określonych) wierszach kolumny k 
        for i in k+1:k+(l-k%l)
            multiplier = A[k, i] / A[k, k]
            A[k, i] = multiplier
            # Iteracja po (określonych) kolumnach w aktualnie eliminowanym wierszu
            # 
            for j in k+1:min(k+l, n)
                A[j, i] -= multiplier * A[j, k]
            end
        end
    end
    return A
end

"""
Funkcja obliczenia wektora X z macierzy A i wektora b metodą rozkładu LU 
(przy użyciu eliminacji Gaussa)
"""
function solve_sparse_matrix_with_lu(A::SparseMatrixCSC{Float64, Int64}, b::Vector{Float64}, 
        n::Int64, l::Int64)

    # Wektor rozwiązania dolnego LU
    sub_b = Vector{Float64}(undef,n)
    ## Rozwiązanie z dolnego LU
    # Iteracja po wierszach
    for i in 1:n
        row_sum = Float64(0.0)
        for j in max(1, Int64(l * floor((i - 1) / l))):i-1
            row_sum += A[j, i] * sub_b[j]
        end
        sub_b[i] = b[i] - row_sum #tutaj A[i,i] = 1, więc brak dzielenia
    end
    # Wektor rozwiązania X
    x = Vector{Float64}(undef,n)
    ## Rozwiązanie wektora X
    # Iteracja po wierszach ("od dołu")
    for i in n:-1:1
        row_sum = Float64(0.0)
        # Iteracja po kolumnach
        for j in i+1:min(n, i+l+1)
            row_sum += A[j, i] * x[j]
        end
        x[i] = (sub_b[i] - row_sum) / A[i, i]
    end
    return x
end



solve_sparse_matrix_with_lu

## 4) Rozklad LU (metodą eliminacji Gaussa z częściowym wyborem)

In [5]:
"""
Funkcja przekształcenia macierzy metodą rozkładu LU 
(przy użyciu eliminacji Gaussa z częściowym wyborem)
"""
function lu_with_choose_on_sparse_matrix(A::SparseMatrixCSC{Float64, Int64}, n::Int64, l::Int64)
    # Wektor permutacji, czyli zamiana wierszy przy częsciowym wyborze, 
    # ale bez "fizycznej" zamiany wierszy w macierzy 
    permutation = Vector{Int64}(undef,n)
    # Podstawowa permutacja
    for i in 1:n
        permutation[i] = i
    end
    # Iteracja po kolumnach
    for k in 1:n-1
        # Maksymalna wartość w danym wierszu
        max_element = abs(A[k, k])
        # Numer wiersza z największą wartością
        max_element_row = k
        for i in k:k+(l-k%l)
            # Znalezienie największej wartości oraz jej numeru wiersza
            if abs(A[k, permutation[i]]) > max_element
                max_element = abs(A[k, permutation[i]])
                max_element_row = i
            end
        end
        # Swap wierszy
        permutation[k], permutation[max_element_row] = permutation[max_element_row], permutation[k]
        # Iteracja po wierszach w kolumnie (tych, które są niezerowe i należy je "usunąć")
        for i in k+1:k+(l-k%l)
            multiplier = A[k, permutation[i]] / A[k, permutation[k]]
            # Przypisanie mnoznika 
            A[k, permutation[i]] = multiplier
            # Iteracja po kolumnach (czyli inaczej odejmowanie wiersza od wiersza)
            for j in k+1:min(k + 2*l, n)
                A[j, permutation[i]] -= multiplier * A[j, permutation[k]]
            end
        end 
    end
    return (A, permutation)
end

"""
Funkcja obliczenia wektora X z macierzy A i wektora b metodą rozkładu LU 
(metodą eliminacji Gaussa z częściowym wyborem)
"""
function solve_sparse_matrix_with_lu_with_choose(A::SparseMatrixCSC{Float64, Int64}, b::Vector{Float64}, 
        n::Int64, l::Int64, permutation)
    # Wektor rozwiązania górnego LU
    sub_b = Vector{Float64}(undef,n)
    ## Rozwiązanie z dolnego LU
    # Iteracja po wierszach
    for i in 1:n
        sum_row = Float64(0.0)
        for j in max(1, Int64(l * floor((i-1) / l)) - 1):i-1
            sum_row += A[j, permutation[i]] * sub_b[j]
        end
        sub_b[i] = b[permutation[i]] - sum_row
    end
    # Wektor rozwiązania X
    x = Vector{Float64}(undef,n)
    ## Rozwiązanie wektora X
    # Iteracja po wierszach ("od dołu")
    for i in n:-1:1
        sum_row = Float64(0.0)
        for j in i+1:min(i + 2*l + 1, n) 
            sum_row += A[j, permutation[i]] * x[j]
        end
        x[i] = (sub_b[i] -  sum_row) / A[i, permutation[i]]
    end
    return x
end

solve_sparse_matrix_with_lu_with_choose

# 2. Sprawozdanie

## Zadanie
W zadaniu należało rozważyć problem rozwiązywania układu równań liniowych $Ax=b$, dla pewnej (nietypowej) macierzy współczynników $A \in \mathbb{R}^{n\times n}$ i wektora prawych stron $b \in \mathbb{R}^{n}$. Macierz A jest rzadką, tj. mającą dużo elementów zerowych, i blokową o następującej strukturze:
<center>
$A=\begin{pmatrix}
A_1 & C_1 & 0 & 0 & 0 & \cdots & 0\\
B_2 & A_2 & C_2 & 0 & 0 & \cdots & 0\\
0 & B_3 & A_3 & C_3 & 0 & \cdots & 0\\
\vdots & \ddots & \ddots & \ddots & \ddots & \ddots & \vdots\\
0 & \cdots & 0 & B_{v-2} & A_{v-2} & C_{v-2} & 0\\
0 & \cdots & 0 & 0 & B_{v-1} & A_{v-1} & C_{v-1}\\
0 & \cdots & 0 & 0 & 0 & B_v & A_v
\end{pmatrix}$,
</center>

$v=\frac{n}{\ell}$, zakładając, że $n$ jest podzielne przez $\ell$, gdzie $\ell$ jest rozmiarem wszystkich kwadratowych macierzy wewnętrznych (bloków): $A_k$, $B_k$ i $C_k$. Mianowicie, $A_k \in \mathbb{R}^{\ell \times \ell}$, $k=1,...,v$ jest macierzą gęstą, $0$ jest kwadratową macierzą zerową stopnia $\ell$, macierz $B_k\in \mathbb{R}^{\ell \times \ell}$, $k=2,...,v$ jest następującej postaci:
<center>
$B_k=\begin{pmatrix}
0 & \cdots & 0 & b^k_1 \\
0 & \cdots & 0 & b^k_2 \\
\vdots &  & \vdots & \vdots \\
0 & \cdots & 0 & b^k_{\ell} \\
\end{pmatrix}$,
</center>

$B_k$ ma tylko jedną, ostatnią kolumnę niezerową. Natomiast $C_k \in \mathbb{R}^{\ell \times \ell}$, $k=1,...,v-1$ jest macierzą diagonalną:
<center>
$C_k=\begin{pmatrix}
c^k_1 & 0 & 0 & \cdots & 0 \\
0 & c^k_2 & 0 & \cdots & 0 \\
\vdots & \ddots & \ddots & \ddots & \vdots \\
0 & \cdots & 0 & c^k_{\ell - 1} & 0 \\
0 & \cdots & 0 & 0 & c^k_{\ell }\\
\end{pmatrix}$.
</center>

## Rozwiązanie
### Eliminacja Gaussa
Jednym ze sposobów rozwiązywania tego typu równań jest użycie metody eliminacji Gaussa. Mając macierz $A$ i wektor prawych stron $b$ można znaleźć rozwiązanie $x$ (wektor) używając podstawowych operacji na macierzy, tj. dodawanie, odejmowanie i mnożenie przez czynnik na wierszach i kolumnach oraz można wiersze i kolumny zamieniać kolejnością. Należy wówczas tak używać tych operacji, aby uzyskać macierz trójkątną górną, gdzie wszystkie elementy pod przekątną macierzy są zerowe.

Przykładowo, mając macierz jak niżej:

<center>
$A=\begin{pmatrix}
a & b & c \\
d & e & f \\
g & h & j \\
\end{pmatrix}$
</center>

należy doprowadzić ją do postaci:

<center>
$A=\begin{pmatrix}
a_1 & b_1 & c_1 \\
0 & e_1 & f_1 \\
0 & 0 & j_1 \\
\end{pmatrix}$.
</center>


Aby powyższą przykładową macierz otrzymać w postaci trójkątnej, należy wykonać następujące kroki (wg algorytmu eliminacji Gaussa):
1. wyeliminować elementy pod elementem $a$ (tj. $d$ i $g$ ):
    - eliminacja $d$: 
        - wyznaczyć mnożnik $m=\frac{d}{a}$
        - od wiersza drugiego (tj. $(d,e,f)$) odjąć wiersz pierwszy (tj. $(a,b,c)$) przemnożony przez mnożnik $m$ (wówczas w drugim wierszu na miejscu elementu $d$ pojawi się $0$)
    - eliminacja $g$
        - wyznaczyć mnożnik $m=\frac{g}{a}$
        - od wiersza trzeciego odjąć wiersz pierwszy przemnożony przez mnożnik $m$ (wówczas w trzecim wierszu na miejscu elementu $g$ pojawi się $0$)
W ten sposób otrzymaliśmy macierz, gdzie wszystkie elementy w kolumnie pierwszej pod przekątną są zerowe
2. wyeliminować elementy pod elementem $e'$ (w tym momencie elementy w wierszu drugim i trzecim są już zmienione):
    - eliminacja $h'$: 
        - wyznaczyć mnożnik $m=\frac{h'}{e'}$
        - od wiersza trzeciego odjąć wiersz drugi przemnożony przez mnożnik $m$.

Czyli w kolejnych krokach otrzymujemy:
<center>
$A=\begin{pmatrix}
a & b & c \\
d & e & f \\
g & h & i \\
\end{pmatrix}$ 
$=>\begin{pmatrix}
a & b & c \\
0 & e' & f' \\
g & h & i \\
\end{pmatrix}$
$=>\begin{pmatrix}
a & b & c \\
0 & e' & f' \\
0 & h' & i' \\
\end{pmatrix}$
$=>\begin{pmatrix}
a & b & c \\
0 & e' & f' \\
0 & 0 & i'' \\
\end{pmatrix}$
</center>.

### Zmodyfikowana eliminacja Gaussa
W przypadku macierzy $A$ z zadania standardowa metoda eliminacji Gaussa jest bardzo niewydajna, gdyż macierz $A$ jest rzadka, a więc znaczna część jej elementów już jest zerowa. Należy więc dostosować ten algorytm, aby przetwarzał tylko elementy niezerowe. W tym celu warto się przyjrzeć przykładowi macierzy. Niech macierz $A$, będzie macierzą z zadania, gdzie $\ell = 3$. "Początek" tej macierzy można przedstawić jako:

<center>
$A=\begin{pmatrix}
\boxed{a^1_{11}} & a^1_{12} & a^1_{13} & c^1_{11} & 0 & 0 & 0 & 0 & 0 & \cdots\\
a^1_{21} & \boxed{a^1_{22}} & a^1_{23} & 0 & c^1_{22} & 0 & 0 & 0 & 0 & \cdots\\
a^1_{31} & a^1_{32} & \boxed{a^1_{33}} & 0 & 0 & c^1_{33} & 0 & 0 & 0 & \cdots\\
0 & 0 & b^2_1 & \boxed{a^2_{11}} & a^2_{12} & a^2_{13} & c^2_{11} & 0 & 0 & \cdots\\
0 & 0 & b^2_2 & a^2_{21} & \boxed{a^2_{22}} & a^2_{23} & 0 & c^2_{22} & 0 & \cdots\\
0 & 0 & b^2_3 & a^2_{31} & a^2_{32} & \boxed{a^2_{33}} & 0 & 0 & c^2_{33} & \cdots\\
0 & 0 & 0 & 0 & 0 & b^3_1 & \boxed{a^3_{11}} & a^3_{12} & a^3_{13} & \cdots\\
0 & 0 & 0 & 0 & 0 & b^3_2 & a^3_{21} & \boxed{a^3_{22}} & a^3_{23} & \cdots\\
0 & 0 & 0 & 0 & 0 & b^3_3 & a^3_{31} & a^3_{32} & \boxed{a^3_{33}} & \cdots\\
\cdots & \cdots & \cdots & \cdots & \cdots & \cdots & \cdots & \cdots & \cdots & \cdots\\
\end{pmatrix}$ &emsp;$(*)$
</center>

(liczby w kwadratach to przekątna macierzy)

Warto tutaj zauważyć dwie właściwości:
1. liczba elementów (niezerowych) pod przekątną tworzą ciąg $(2,1,3,2,1,3,2,1,...,0)$,
2. w każdym kroku (w trakcie działania algorytmu) "na prawo" od przekątnej są dokładnie 3 elementy niezerowe (np. w pierwszej iteracji "na prawo" mamy elementy $(a^1_{12}, a^1_{13}, c^1_{11})$; w drugiej iteracji - już po odjęciu od wiersza drugiego wiersz pierwszy - mamy elementy $(a^{1'}_{23}, -c^1_{11}*m_1, c^1_{22})$ ).

Z powyższych właściwości na powyższym przykładzie, można stworzyć zależności ogólne dla liczb $n$ i $\ell$ z zadania i gdzie $k$ jest numerem kolumny (również numerem kolejnej iteracji algorytmu) :
1. liczba elementów (niezerowych) do eliminacji pod przekątną tworzą ciąg, który można opisać wzorem $l-(k\mod l)$, dla $k=1,...,n$
2. numery kolumn elementów niezerowych "na prawo" od przekątnej (w kolejnych iteracjach algorytmu) są w przedziale od $k+1$ do $k+l$ (lub "do $n$", gdy $k+l>n$)

#### Implementacja

Implementację powyższego algorytmu można przedstawić poniższym pseudokodem (gdzie $A[i,j]$ oznacza odwołanie się do elementu macierzy $A$ na $i$-tym wierszu, $j$-tej kolumnie):

![](gauss_function.png)

#### Złożoność czasowa
(przyjmujemy, że odczyt z tablicy *SparseArray* jest w czasie stałym)

Algorytm składa się z dwóch głównych pętli. Pierwsza z nich wykonuje się $n$ razy. Ponadto zawiera ona pętlę, która wykonuje się co najwyżej $\ell$ razy oraz jeszcze jedna pętla w środku tej poprzedniej, która również wykonuje się co najwyżej $\ell$ razy. Stąd złożoność pierwszej pętli jest nie większa niż $O(n)$

Druga główna pętla również wykonuje się $n$ razy. Dodatkowo zawiera ona pętlę wykonującą się co najwyżej $\ell$ razy - stąd również wychodzi złożoność $O(n)$. Jako, że reszta operacji jest wykonywana w czasie stałym, to złożoność całego algorytmu: $O(n)$.

### Eliminacja Gaussa z częściowym wyborem
W przypadku wcześniejszego algorytmu można zauważyć pewien problem w trakcie jego obliczeń. Mianowicie, przy wyliczaniu mnożnika $m$ mamy dzielenie, np. $m=\frac{a^1_{21}}{a^1_{11}}$ dla pierwszego elementu we wcześniejszym przykładzie macierzy $A$ $(*)$. Stąd w trakcie działania algorytmu może dojść sytuacji, że będzie dzielenie przez $0$, gdy jedna z wartości na przekątnej macierzy jest zerem. W celu uniknięcia takiego problemu powstała lekko zmodyfikowana wersja algorytmu nazwana eliminacją Gaussa z częściowym wyborem.

Polega ona na wykonaniu dodatkowej operacji (przed przystąpieniem do eliminacji elementów w danej kolumnie) zamiany wierszy: wykonywana jest zamiana wiersza, który w danej iteracji (po kolumnach) zawiera element na przekątnej, z wierszem (poniżej przekątnej danej kolumy), którego element (w tej samej kolumnie) ma największą wartość (z dokładnością do wartości bezwględnej). 

Ilustrując to na przykładzie:
<center>
$A=\begin{pmatrix}
a & b & c & d & \cdots\\
e & f & g & h & \cdots\\
i & j & p & r & \cdots\\
q & s & t & u & \cdots\\
\cdots & \cdots & \cdots & \cdots & \cdots\\
\end{pmatrix}$
</center>.

Załózmy, że mamy powyższą macierz. Zanim się przystapi do eliminacji elementu $e$ (oraz kolejnych w pierwszej kolumnie) należy wykonać dodatkową operację. Spośród wierszy od elementu $a$ (który leży na przekątnej) w dół, wybieramy wiersz, którego element w kolumnie pierwszej zawiera największą wartość bezwględną - czyli $\max(|a|,|e|, |i|, |q|,...)$. Następnie zamieniane są miejscami wiersz pierwszy (ten zawierający element $a$ na przekątnej) z wierszem, który zawiera właśnie wyliczoną wartość maksymalną. Następnie kontynuowany jest algorytm eliminacji Gaussa. 

Przy zamianie wierszy należy również pamiętać, aby zamieniać odpowiednio elementy wektora prawych stron.

### Zmodyfikowana eliminacja Gaussa z częściowym wyborem
W przypadku macierzy $A$ z zadania również ten dodatkowy etap zamiany wierszami można zmodyfikować, by dostosować go do specyficznej postaci macierzy. Modyfikacji tutaj podlegają dwie rzeczy:
1. liczba wierszy sprawdzanych "w dół" pod elementem na przekątnej macierzy,
2. liczba elementów w wierszach, które należy zamienić.

Ad 1. Z poprzedniego algorytmu (eliminacji Gaussa) wiemy, że liczba elementów niezerowych pod elementem na przekątnej macierzy jest dokładnie $l-(k \mod l)$ (gdzie $k$ to numer kolumny). Stąd wniosek, że kandydatami na zamianę są tylko wiersze, które mieszczą się w przedziale $[k, k + l-(k \mod l)]$.

Ad 2. W przypadku tak specyficznej macierzy, należy ograniczyć liczbę elementów, które należy przestawić (oraz zmodyfikować przedział elementów, które należy odejmować w trakcie odejmowania wierszy), gdyż również tutaj znaczna część z nich to elementy zerowe. W momencie zamiany wierszy warto zauważyć, że "na lewo" od elementu na przekątnej macierzy wszystkie elementy są już zerowe - również dla wierszy poniżej. Stąd wniosek, że w implementacji algorytmu należy zamieniać tylko elementy w kolumnach "na prawo" od elementu na przekątnej. Przyglądając się macierzy $A$ $(*)$ można zauważyć, że ostatni niezerowy element w każdym wierszu należy do bloku macierzy $C_k$. Elementy z bloku macierzy $C_k$ są oddalone od elementu, który leży na przekątnej, dokładnie o odgległość $\ell$.<br/>
Z faktu (1.) wiadomo również, że maksymalna odgległość między zamienianymi wierszami wynosi $\ell$. Jest to skrajny przypadek. Załóżmy, że dla pewnej macierzy rozpatrujemy taki skrajny przypadek, w którym zamieniamy miejscami wiersz $\ell$ i wiersz $2\ell$. Niech $c_1$ oznacza element w wierszu $\ell$ z bloku macierzy $C_k$ oraz analogicznie oznaczmy $c_2$ dla wiersza $2\ell$. Wówczas element $c_1$ jest w kolumnie $2\ell$, a element $c_2$ jest w kolumnie $3\ell$. Wówczas można zauważyć, że po zamianie powyższych dwóch wierszy, ostatni element niezerowy w wierszu $\ell$ będzie w kolumnie $3\ell$. Stąd wniosek, że w najbardziej skrajnym przypadku, odgległość w wierszu między elementem na przekątnej macierzy, a ostatnim elementem niezerowym "na prawo" od przekątnej wynosi $3\ell-\ell=2\ell$.

Powyższe dwa fakty zostaną wykorzystane w implementacji algorytmu.

#### Implementacja
Implementację powyższego algorytmu można przedstawić poniższym pseudokodem (gdzie 𝐴[𝑖,𝑗] oznacza odwołanie się do elementu macierzy 𝐴 na 𝑖-tym wierszu, 𝑗-tej kolumnie). 
<br/>
W implementacji algorytm nie dokonuje "fizycznej" zamiany wierszami przestawiając wszystkie elementy między wierszami. Zamiast tego, została wykorzystana dodatkowa tablica ($p$), która przetrzymuje informacje o permutacji wierszy, tj. początkowo tablica wypełniona jest wartościai $(1,2,...,n)$. W momencie zamiany wierszy dokonuje się tylko zamiany dwóch wartości w owej tablicy, utrzymując w ten sposób aktualną tablicę permutacji wierszy macierzy. Wówczas, zamiast odwoływać się do elementu tablicy używając adnotacji $A[i,j]$, algorytm odwołuje się do elementu adnotacją $A[p[i], j]$.

![](gauss_choose_function.png)

#### Złożoność czasowa
Algorytm ma bardzo podobną strukturę jak poprzedni, jednakże zawiera tylko dodatkową pętlę w pierwszej głównej pętli (która wykonuje się co najwyżej $\ell$ razy). Stąd złożoność tego algorytmu również jest $O(n)$.

### Rozkład LU
Drugą metodą rozwiązania równania $Ax=b$ jest metoda rozkładu $LU$. Polega ona na rozłożeniu macierzy $A$ na iloczyn dwóch innych macierz $A=LU$, gdzie $L$ jest macierzą trójkątną dolną, a $U$ - macierzą trójkątną górną. Aby rozkład na dwie macierze był jednoznaczy, przyjmuje się, że elementy na przekątnej macierzy $L$ mają wartości $1$. Przykład takiego rozkładu:
<center>
$A=\begin{pmatrix}
a_{11} & a_{12} & a_{13} & a_{14}\\
a_{21} & a_{22} & a_{23} & a_{24}\\
a_{31} & a_{32} & a_{33} & a_{34}\\
a_{41} & a_{42} & a_{43} & a_{44}\\
\end{pmatrix}=>$
$L=\begin{pmatrix}
1 & 0 & 0 & 0\\
l_{21} & 1 & 0 & 0\\
l_{31} & l_{32} & 1 & 0\\
l_{41}& l_{42} & l_{43} & 1\\
\end{pmatrix}$;
$U=\begin{pmatrix}
u_{11} & u_{12} & u_{13} & u_{14}\\
0 & u_{22} & u_{23} & u_{24}\\
0 & 0 & u_{33} & u_{34}\\
0 & 0 & 0 & u_{44} \\
\end{pmatrix}$;
</center>.

Wówczas układ równań przyjmuje postać: 

<center>$L * U * x = b$,<center/>

a jego rozwiązanie sprowadza się do rozwiązania dwóch układów równań z macierzami trójkątnymi:
       <center>$L * b' = b$<br/>$U * x = b'$<center/>
           
Aby otrzymać rozkład na macierze $LU$ można użyć przedstawionych wcześniej algorytmów eliminacji Gaussa bez- oraz z częściowym wyborem z niewielkimi modyfikacjami. Eliminacja Gaussa pozwala na uzyskanie macierzy trójkątnej górnej $U$, zaś macierz trójkątną dolną uzyskamy z mnożników, które są obliczane w trakcie działania tego algorytmów, tj. każdemu elementowi $l_{ij}$ poniżej przekątnej macierzy $L$ przypisuje się wartość mnożnika $m_{ij}$, który był wyliczony w momencie eliminacji elementu $a_{ij}$ macierzy $A$ (oraz elementom przekątnej macierzy $L$ przypisuje się wartość $1$).

#### Implementacja
Jako, że powyższa metoda wymaga użycia już przedstawionych algorytmów eliminacji Gaussa, dlatego poniższe algorytmy są tak naprawdę niewielkimi modyfikacjami wcześniejszych. Istotne różnice:
1. elementy w macierzy nie będą zerowane, zaś będzie przypisywany im ich mnożnik,
2. wektor prawych stron $b$ nie będzie modyfikowany,
3. rozwiązanie jest roszerzone o dodatkową pętle, która rozwiązuje układ równań z macierzą trójkątną dolną.

Ad 1. Mało efektywnym rozwiązaniem jest utrzymywanie dwóch macierzy $L$ i $U$ - zwłaszcza, gdy macierz $A$ z zadania jest macierzą rzadką. Zamiast tego w implementacji jest wykorzystywana tylko jedna macierz. Wszystkie elementy dwóch macierzy $L$ i $U$ leżą na przekątnych oraz (odpowiednio) poniżej lub powyżej nich oraz ponadto macierz $L$ na przekątnej ma elementy, których wartość wynosi $1$. Wówczas obie macierze można trzymać w jednej, gdyż elementy nieleżące na przekątnych się nie pokrywają między macierzami oraz z punktu widzenia algorytmu, istotne jest tylko pamiętanie przekątnej macierzy $U$. Stąd w jednej macierzy są trzymane wartości macierzy $U$ (przekątna i elementy powyżej niej, które powstały w wyniku eliminacji Gaussa) oraz wartości macierzy $L$ (elementy poniżej przekątnej, której wartości to są odpowiednie mnożniki).

Ad 2. Efektem działania metody rozkładu $LU$ jest rozbicie macierzy $A$ na iloczyn dwóch innych. Jednakże, po wymnożeniu macierzy $LU$ otrzymuje się macierz $A$ w takiej samej postaci, jak była na początku. Stąd wektor prawych stron nie jest aktualizowany w trakcie działania, gdyż sama macierz $A$ nie zmienia swojej struktury.

Ad 3. Dzięki rozkładowi $LU$ otrzymujemy dwa układy równań zamiast jednego. Stąd algorytmy są rozszerzone o dodatkową pętlę rozwiązującą dodatkowy układ równań.

![](lu_function.png)

![](lu_choose_function.png)

#### Złożoności czasowe
Oba powyższe algorytmy są drobnymi modyfikacjami wcześniejszych algorytmów. Poza zmianami w operacjach, których czas jest stały, to zostały dodane do nich tylko jedne "główne" pętle, które są kopią operacji odpowiedzialnej za obliczenie układu równań - czyli ich złożoność jest również nie większa niż $O(n)$. Stąd złożoności obu algorytmów również są $O(n)$.

# 3. Testy

## Test manualny
"Okienko" do własnego testowania pojedynczej macierzy z pliku.

In [ ]:
# Metoda rozwiązania: 
#1 - eliminacja Gaussa, 
#2 - eliminacja Gaussa z wyborem, 
#3 - rozkład LU, 
#4 - rozkład LU z wyborem
solve_type = 1
# Czy wczytać wektor z pliku: 
# true - wektor zostanie wczytany z pliku, 
# false - wektor zostanie wygenerowany
do_load_b_vector_from_file = false
# nazwa pliku macierzy
matrix_file_name = "16x16A.txt"
# nazwa pliku wektora, jesli wczytywany z pliku
vector_file_name = "16x16b.txt"
# nazwa pliku do zapisania wyniku
save_file_name = "manual_test_result.txt"


#### Rozwiazywanie
# Wczytanie macierzy
A, n, l = load_matrix_from_file(matrix_file_name)
b = nothing
x = nothing
# Wczytanie wektora (lub wygenerowanie)
if do_load_b_vector_from_file
    b, n2 = load_b_from_file(vector_file_name)
else
    b = generate_b_vector(A, n, l)
end
# Obliczenie x
if solve_type == 1
    x = solve_sparse_matrix_with_gauss_elimination(A, b, n, l)
elseif solve_type == 2
    x = solve_sparse_matrix_with_gauss_elimination_with_choose(A, b, n, l)
elseif solve_type == 3
    tmp = lu_on_sparse_matrix(A, n, l)
    x = solve_sparse_matrix_with_lu(tmp, b, n, l)
else
    tmp, perm = lu_with_choose_on_sparse_matrix(A, n, l)
    x = solve_sparse_matrix_with_lu_with_choose(tmp, b, n, l, perm)
end
# Zapisanie i wyświetlenie
write_results_to_file(save_file_name, x, n, !do_load_b_vector_from_file)
print(x)

#Czyszczenie
x = nothing
A = nothing
b = nothing
n = nothing
l = nothing
tmp = nothing
perm = nothing
is_b_generated = nothing

## Test poprawności
W poniższym teście przy pomocy modułu $Test$ i funkcji $isapprox()$ dla każdej metody rozwiązania równania $Ax=b$ sprawdzane jest, czy rozwiązanie $x$ jest "w pobliżu" wektora oczekiwanego $(1,...,1)$ - dla danych testowych macierzy $A$ i wektora prawych stron $b$ z zadania.

In [33]:
using Test
test_names = ["Macierz 16 x 16", "Macierz 10 000 x 10 000", "Macierz 50 000 x 50 000"]
A_files_name = ["16x16A.txt", "10000x10000A.txt", "50000x50000A.txt"]
b_files_name = ["16x16b.txt", "10000x10000b.txt", "50000x50000b.txt"]
model_result = [ones(16), ones(10000), ones(50000)]
println("Rozpoczęto testy")
for i in 1:length(test_names)
    @testset "$(test_names[i])" begin

        A, n, l = load_matrix_from_file(A_files_name[i])
        b, n2 = load_b_from_file(b_files_name[i])

        @test isapprox(solve_sparse_matrix_with_gauss_elimination(A, b, n, l), model_result[i])

        A, n, l = load_matrix_from_file(A_files_name[i])
        b, n2 = load_b_from_file(b_files_name[i])
        @test isapprox(solve_sparse_matrix_with_gauss_elimination_with_choose(A, b, n, l),  model_result[i])

        A, n, l = load_matrix_from_file(A_files_name[i])
        b, n2 = load_b_from_file(b_files_name[i])
        tmp = lu_on_sparse_matrix(A, n, l)
        @test isapprox(solve_sparse_matrix_with_lu(tmp, b, n, l),  model_result[i])

        A, n, l = load_matrix_from_file(A_files_name[i])
        b, n2 = load_b_from_file(b_files_name[i])
        tmp, perm = lu_with_choose_on_sparse_matrix(A, n, l)
        @test isapprox(solve_sparse_matrix_with_lu_with_choose(tmp, b, n, l, perm),  model_result[i])
    end
end
println("Zakończono testy")

Rozpoczęto testy
Test Summary:   | Pass  Total
Macierz 16 x 16 |    4      4
Test Summary:           | Pass  Total
Macierz 10 000 x 10 000 |    4      4
Test Summary:           | Pass  Total
Macierz 50 000 x 50 000 |    4      4
Zakończono testy


## Test wydajności
Na podstawie pewnego zestawu danych testowych zostały przeprowadzaone testy wydajności - liczby operacji, wykorzystanej pamięci oraz czasu wykonywania - na wszystkich wcześniejszych implementacjach algorytmów.

In [45]:
using Plots
# Wartości początkowe
test_size = [1000,2000,4000,8000,16000,20000,30000,40000]
l_size = 10

## Testowanie eliminacji Gaussa
time_gauss_elimination = Vector{Float64}(undef, length(test_size))
memory_gauss_elimination = Vector{Float64}(undef, length(test_size))
operations_gauss_elimination = Vector{Float64}(undef, length(test_size))
for i in 1:length(test_size)
    # Generowanie macierzy
    matrixgen.blockmat(test_size[i], l_size, 1.0, "test.txt")
    # Załadowanie macierzy
    A, n, l = load_matrix_from_file("test.txt")
    # Wygenerowanie wektora b
    b = generate_b_vector(A, n, l)
    
    # Wyniki
    result = @timed solve_sparse_matrix_with_gauss_elimination(A, b, n, l)
    time_gauss_elimination[i] = result[2] 
    memory_gauss_elimination[i] = result[3]
    operations_gauss_elimination[i] = test_size[i] * l_size^2 + 2 * test_size[i]
end

## Testowanie eliminacji Gaussa z wyborem
time_gauss_elimination_choose = Vector{Float64}(undef, length(test_size))
memory_gauss_elimination_choose = Vector{Float64}(undef, length(test_size))
operations_gauss_elimination_choose = Vector{Float64}(undef, length(test_size))
for i in 1:length(test_size)
    # Generowanie macierzy
    matrixgen.blockmat(test_size[i], l_size, 1.0, "test.txt")
    # Załadowanie macierzy
    A, n, l = load_matrix_from_file("test.txt")
    # Wygenerowanie wektora b
    b = generate_b_vector(A, n, l)

    #Wyniki
    results = @timed solve_sparse_matrix_with_gauss_elimination_with_choose(A, b, n, l)
    time_gauss_elimination_choose[i] = results[2]
    memory_gauss_elimination_choose[i] = results[3]
    operations_gauss_elimination_choose[i] = test_size[i] * l_size^3 + 2 * test_size[i]
end


## Testowanie rozkladu LU
time_lu = Vector{Float64}(undef, length(test_size))
memory_lu = Vector{Float64}(undef, length(test_size))
operations_lu = Vector{Float64}(undef, length(test_size))

time_lu_solve = Vector{Float64}(undef, length(test_size))
memory_lu_solve = Vector{Float64}(undef, length(test_size))
operations_lu_solve = Vector{Float64}(undef, length(test_size))
for i in 1:length(test_size)
    # Generowanie macierzy
    matrixgen.blockmat(test_size[i], l_size, 1.0, "test.txt")
    # Załadowanie macierzy
    A, n, l = load_matrix_from_file("test.txt")
    # Wygenerowanie wektora b
    b = generate_b_vector(A, n, l)

    # Wyniki
    results = @timed lu_on_sparse_matrix(A, n, l)
    A_r = results[1]
    time_lu[i] = results[2]
    memory_lu[i] = results[3]
    operations_lu[i] = test_size[i] * (l_size^2 + l_size)

    results = @timed solve_sparse_matrix_with_lu(A_r, b, n, l)
    time_lu_solve[i] = results[2] + time_lu[i]
    memory_lu_solve[i] = results[3] + memory_lu[i]
    operations_lu_solve[i]  = test_size[i] * (l_size^2 + l_size) + 2 * test_size[i] * l_size
end

## Testowanie rozkladu LU z wyborem
time_lu_choose = Vector{Float64}(undef, length(test_size))
memory_lu_choose = Vector{Float64}(undef, length(test_size))
operations_lu_choose = Vector{Float64}(undef, length(test_size))

time_lu_solve_choose = Vector{Float64}(undef, length(test_size))
memory_lu_solve_choose = Vector{Float64}(undef, length(test_size))
operations_lu_solve_choose = Vector{Float64}(undef, length(test_size))
for i in 1:length(test_size)
    # Generowanie macierzy
    matrixgen.blockmat(test_size[i], l_size, 1.0, "test.txt")
    # Załadowanie macierzy
    A, n, l = load_matrix_from_file("test.txt")
    # Wygenerowanie wektora b
    b = generate_b_vector(A, n, l)

    results = @timed lu_with_choose_on_sparse_matrix(A, n, l)
    A_r, perm = results[1]
    time_lu_choose[i] = results[2]
    memory_lu_choose[i] = results[3]
    operations_lu_choose[i]  = test_size[i] * (l_size^3 + l_size)

    results = @timed solve_sparse_matrix_with_lu_with_choose(A_r, b, n, l, perm)
    time_lu_solve_choose[i] = results[2] + time_lu_choose[i]
    memory_lu_solve_choose[i] = results[3] + memory_lu_choose[i]
    operations_lu_solve_choose[i] = test_size[i] * (l_size^3 + l_size) + 2 * test_size[i] * l_size
end

## Generowanie wykresu - czasy
times = [time_gauss_elimination, time_gauss_elimination_choose, time_lu_solve, time_lu_solve_choose]
plot(test_size, times, label=["Gauss" "Gauss z wyborem" "LU" "LU z wyborem"], 
    linewidth=1.0, title="Czas rozwiązania równania Ax=b")
xlabel!("Rozmiar macierzy")
ylabel!("Czas")
savefig("time_gauss_lu.png")
## Generowanie wykresu - pamięć

memory=[memory_gauss_elimination, memory_gauss_elimination_choose, memory_lu_solve, memory_lu_solve_choose]
plot(test_size, memory, label=["Gauss" "Gauss z wyborem" "LU" "LU z wyborem"], 
    linewidth=1.0, title="Pamięć zużyta podczas rozwiązywania równania Ax=b")
xlabel!("Rozmiar macierzy")
ylabel!("Wykorzystana pamięć")
savefig("memory_gauss_lu.png")
## Generowanie wykresu - operacje

operations=[operations_gauss_elimination,operations_gauss_elimination_choose,
    operations_lu_solve,operations_lu_solve_choose]
plot(test_size, operations, label=["Gauss" "Gauss z wyborem" "LU" "LU z wyborem"], 
    linewidth=1.0, title="Operacje wykonane podczas rozwiązywania równania Ax=b")
xlabel!("Rozmiar macierzy")
ylabel!("Liczba operacji")
savefig("operations_gauss_lu.png")

### Tylko LU i LU z wyborem
## Generowanie wykresu - czas

time=[time_lu,time_lu_choose]
plot(test_size, time, label=["LU" "LU z wyborem"], 
    linewidth=1.0, title="Czas rozkładu na LU")
xlabel!("Rozmiar macierzy")
ylabel!("Czas")
savefig("time_lu.png")

## Generowanie wykresu - pamięć

memory=[memory_lu, memory_lu_choose]
plot(test_size, memory, label=["LU" "LU z wyborem"], 
    linewidth=1.0, title="Zużyta pamięć do rozkładu LU")
xlabel!("Rozmiar macierzy")
ylabel!("Wykorzystana pamięć")
savefig("memory_lu.png")

## Generowanie wykresu - operacje

operations = [operations_lu, operations_lu_choose]
plot(test_size, operations, label=["LU" "LU z wyborem"], 
    linewidth=1.0, title="Operacje wykonane podczas rozkładu LU")
xlabel!("Rozmiar macierzy")
ylabel!("Liczba operacji")
savefig("operations_lu.png")

### Liczba operacji

![](operations_gauss_lu.png) ![](operations_lu.png)

Na powyższych wykresach zostały przedstawione zależności między liczbą operacji wykonaną przez algorytmy a rozmiarem tablicy. Przy założeniu, że dostęp do $SparseArray$ jest stał, widać, że algorytmy mają złożoność $O(n)$. Widać również, że algorytmy z częściowym wyborem wykonują więcej operacji, niż te bez; oraz, że algorytmy z rozkładem $LU$ wykonują więcej operacji, niż odpowiadające im algorytmy z samą eliminacją Gaussa.

### Wykorzystana pamięć

![](memory_gauss_lu.png) ![](memory_lu.png) 

Na powyższych wykresach widać, że $SparseArray$ jest w pewien sposób optymalizowana pod względem wykorzystania pamięci i więcej elementów niekoniecznie oznacza większą zaalokowaną przestrzeń na przechowanie danych.

### Czas działania
![](time_gauss_lu.png) ![](time_lu.png)

W przypadku wykresów czasu działania zaobserwować można, że rozbiegają się one z wykresami liczby operacji poz względem złożoności obliczeniowej. Wykresy liczby operacji wskazują na złożoność liniową, zaś wykresy czasu wskazują na złożoność kwadratową.

### Wnioski
Z powyższych obserwacji wywnioskować można, że dostosowanie standardowych algorytmów do specyficznej postaci macierzy z zadania jest zdecydowanie opłacalne. Standardowe algorytmy mają złożoność sześcienną $O(n^3)$, a po zmodyfikowaniu ich otrzymano algorytmy o złożoności nawet liniowej $O(n)$ (przy założeniu o stałym dostępie do $SparseArray$), co jest zdecydowaną oszczędnością zasobów.

Wywnioskować również można, że założenie o stałym dostępie do elemntów tablicy $SparseArray$ jest błędne. Zaobserować to można w rozbieżności między wykresami liczby operacji oraz wyrkesami czasu. Wykresy liczby operacji wskazują na złożoność liniową, zaś wykresy czasu wskazują na złożoność kwadratową. Stąd wniosek, że założenie o stałym dostępie do elementów tablicy $SparseArray$ jest błędne.